In [6]:
import logging
from collections import deque
import math
 
class Trans:
    datetime=None
    amount=None
    price=None
 
    def __init__(self, datetime, amount, price):
        self.datetime=datetime
        self.amount=amount
        self.price=price
     
    def getInfo(self):
        return(str(self.datetime)+"; "+
                str(self.amount)+"; "+
                str(self.price))+"; "
 
def balanceFifo(all_trans):
 
    qTransactions = deque() 
 
    for t in all_trans:
        #Add first element to the queue
        if len(qTransactions)==0:
            logging.debug('Added the first element: %s',t.getInfo())
            qTransactions.append(t)
            continue
 
        while (t.amount!=0 and len(qTransactions)>0):
            #investigate the first element from the queue
            tq=qTransactions.popleft()
            #the same type of transaction: both sell or both buy
            if tq.amount*t.amount>0:
                #return the first element back to the same place
                qTransactions.appendleft(tq)
                #add the new element to the list
                qTransactions.append(t)
                logging.debug('Added: %s',t.getInfo())
                break
             
            #contrary transactions: (sell and buy) or (buy and sell) 
            if tq.amount*t.amount<0:
                logging.debug('Transaction : %s',t.getInfo())
                logging.debug('... try to balance with: %s',tq.getInfo())
 
                #The element in the queue have more units and takes in the current transaction
                if abs(tq.amount)>abs(t.amount):
                    insertTransaction(tq.datetime,t.datetime,\
                            math.copysign(t.amount,tq.amount), tq.price,t.price)
                    #update the amount of the element in the queue
                    tq.amount=tq.amount+t.amount
                    #return the element back to the same place
                    qTransactions.appendleft(tq)
                    logging.debug('Removed transaction: %s',t.getInfo())
                    #the transaction has been balanced, take a new transaction
                    break
                 
                #The element from the queue and transaction have the same amount of units
                if abs(tq.amount)==abs(t.amount):
                    insertTransaction(tq.datetime,t.datetime,\
                            math.copysign(t.amount,tq.amount), tq.price,t.price)
                     
                    #update the amount in the transaction 
                    t.amount=0
                    logging.debug('Balanced, removed transaction: %s',t.getInfo())
                    logging.debug('Balanced, removed from the queue: %s',tq.getInfo())
                    #the transaction has been balanced, take a new transaction
                    continue
                #The transaction has more units
                if abs(tq.amount)<abs(t.amount):
                    #update the units in transaction, (remove element from the queue)
                    t.amount=t.amount+tq.amount
                    insertTransaction(tq.datetime,t.datetime,tq.amount,tq.price,t.price)
                    logging.debug('Removed from queue: %s',tq.getInfo())
                     
                    #the transaction has not been balanced, 
                    #take a new element from the queue (t.amount>0)
                    continue
                 
        #We have unbalanced transaction but the queue is empty            
        if (t.amount!=0 and len(qTransactions)==0):
            #Add unbalanced transaction to the queue
            #The queue changes polarisation
            qTransactions.append(t)
            logging.debug('Left element: %s',t.getInfo())
     
     
    #If something remained in the queue, treat it as open or part-open transactions
    while (len(qTransactions)>0):
        tq=qTransactions.popleft()
        logging.debug('Remained on list transaction: %s',tq.getInfo())
 
def insertTransaction(dateStart,dateEnd,amount,priceStart,priceEnd):
     
    print("Bought={}, sold={},  amount={}, buy price={}, sell_price={}, gain={}".\
            format(dateStart,dateEnd,amount,priceStart,priceEnd, amount*(priceEnd-priceStart)))
 
#Uncomment if you want to see more information
logging.basicConfig(level=logging.DEBUG)

In [37]:
trans_list=list()
trans = Trans[["2022-01-01", 5, 10], ["2022-01-02",5,12], ["2022-01-03",-5,12], ["2022-01-04",-5,14]]
trans_list.append(trans)
balanceFifo(trans_list)

TypeError: list indices must be integers or slices, not tuple

In [29]:
#Case 1
print("Case 1")
trans=Trans("2022-01-01",5,10)
trans_list.append(trans)
trans=Trans("2022-01-02",5,12)
trans_list.append(trans)
trans=Trans("2022-01-03",-5,12)
trans_list.append(trans)
trans=Trans("2022-01-04",-5,14)
trans_list.append(trans)
trans_list
balanceFifo(trans_list)
trans_list.clear()

Case 1
Bought=2022-01-01, sold=2022-01-03,  amount=5.0, buy price=10, sell_price=12, gain=10.0
Bought=2022-01-02, sold=2022-01-04,  amount=5.0, buy price=12, sell_price=14, gain=10.0


In [30]:
#Case 2
print("Case 2")
trans=Trans("2022-01-01",5,10)
trans_list.append(trans)
trans=Trans("2022-01-02",5,12)
trans_list.append(trans)
trans=Trans("2022-01-03",-2,12)
trans_list.append(trans)
trans=Trans("2022-01-04",-8,14)
trans_list.append(trans)
trans_list
balanceFifo(trans_list)
trans_list.clear()

Case 2
Bought=2022-01-01, sold=2022-01-03,  amount=2.0, buy price=10, sell_price=12, gain=4.0
Bought=2022-01-01, sold=2022-01-04,  amount=3, buy price=10, sell_price=14, gain=12
Bought=2022-01-02, sold=2022-01-04,  amount=5.0, buy price=12, sell_price=14, gain=10.0


In [31]:
trans_list.clear()
#Case 3
print("Case 3")
trans=Trans("2022-01-01",5,10)
trans_list.append(trans)
trans=Trans("2022-01-02",5,12)
trans_list.append(trans)
trans=Trans("2022-01-03",-8,12)
trans_list.append(trans)
trans=Trans("2022-01-04",-2,14)
trans_list.append(trans)
balanceFifo(trans_list)
trans_list.clear()

Case 3
Bought=2022-01-01, sold=2022-01-03,  amount=5, buy price=10, sell_price=12, gain=10
Bought=2022-01-02, sold=2022-01-03,  amount=3.0, buy price=12, sell_price=12, gain=0.0
Bought=2022-01-02, sold=2022-01-04,  amount=2.0, buy price=12, sell_price=14, gain=4.0
